# Mobi Vancouver Bike Share Data

Download and process complete history (2018-2025, ~7.6M trips).

Runs on Databricks Unity Catalog (Serverless or cluster).

In [0]:
%pip install requests pandas pyarrow beautifulsoup4 openpyxl mlflow markdownify

%restart_python

In [0]:
# Notebook bootstrap: autoreload, src path, imports
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Add the repository's src directory using a relative path
src_path = Path.cwd() / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Common imports
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import mlflow

# Project imports
from src.mobi import (
    fetch_station_info_from_gbfs,
    fetch_station_status_from_gbfs,
    BasicSiteScraper,
    download_all_trip_data,
    combine_trip_data,
    save_to_parquet,
)

In [0]:
%sql
SELECT CURRENT_METASTORE();

In [0]:
# Project config & UC paths
config = mlflow.models.ModelConfig(development_config='config.yaml')
dbutils.widgets.text('catalog', config.get('catalog'))
dbutils.widgets.text('schema', config.get('schema'))
dbutils.widgets.text('volume', config.get('raw_data_vol') or 'mobi_data')
dbutils.widgets.text('shuffle_partitions', '64')

catalog = dbutils.widgets.get('catalog')
schema = dbutils.widgets.get('schema')
volume = dbutils.widgets.get('volume')
shuffle_partitions = dbutils.widgets.get('shuffle_partitions')

from pathlib import Path

# Ensure catalog, schema, and volume exist; create if missing
# TODO: add check for catalog
spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{catalog}`.`{schema}`")
spark.sql(f"CREATE VOLUME IF NOT EXISTS `{catalog}`.`{schema}`.`{volume}`")

# UC Volume root (used by pandas and Spark)
volume_root = Path('/Volumes') / catalog / schema / volume
trip_data_dir = volume_root / 'trip_data'
site_data_dir = volume_root / 'mobi_site'

# Spark tuning (shared across all writes)
spark.conf.set('spark.sql.shuffle.partitions', shuffle_partitions)

print('Using UC Volume storage:')
print(f"  catalog.schema: {catalog}.{schema}")
print(f"  volume: {volume}")
print(f"  root: {volume_root}")

## Download Trip Data

In [0]:
# Download all files
print("Downloading trip data...")
files = download_all_trip_data(trip_data_dir / "raw")
print(f"Downloaded {len(files)} files")

# Process and combine
print("\nProcessing...")
trips_df = combine_trip_data(files)
print(f"Total trips: {len(trips_df):,}")

# Save
save_to_parquet(trips_df, trip_data_dir / "mobi_trips.parquet")

## Download Station Data

In [0]:
# Fetch from GBFS API
print("Fetching stations...")
stations = fetch_station_info_from_gbfs()
status = fetch_station_status_from_gbfs()

# Combine
stations = stations.merge(status, on="station_id", how="left")
print(f"Total stations: {len(stations)}")

# Save
save_to_parquet(stations, trip_data_dir / "mobi_stations.parquet")
stations.to_csv(trip_data_dir / "mobi_stations.csv", index=False)

## Scrape mobi Webiste

In [0]:
from mobi import BasicSiteScraper

# Configure scraper
BASE_URL = "https://www.mobibikes.ca/en/"
START_URL = BASE_URL

scraper = BasicSiteScraper(
    base_url=BASE_URL,
    delay=0.75,      # seconds between requests
    max_depth=1,     # 0 = only start page; 1 = start + its links
)

# Run the scrape
pages = scraper.scrape_recursive(START_URL)

print(f"Pages scraped: {len(pages)}")
# Preview a few URLs
list(pages.keys())[:10]


In [0]:
# Save markdown content per page
from urllib.parse import urlparse
import re

raw_site_data_dir = site_data_dir / "raw"
raw_site_data_dir.mkdir(parents=True, exist_ok=True)  # Ensure 'raw' directory exists


def url_to_filename(url: str) -> str:
    parsed = urlparse(url)
    path = parsed.path.rstrip("/") or "index"
    safe = re.sub(r"[^a-zA-Z0-9._-]+", "_", path)
    if not safe.endswith(".md"):
        safe += ".md"
    return safe

count = 0
for url, data in pages.items():
    filepath = raw_site_data_dir / url_to_filename(url)
    filepath.write_text(data.get("content", ""), encoding="utf-8")
    count += 1

print(f"Wrote {count} markdown files to {raw_site_data_dir}")

# Save to parquet
records = []
for url, data in pages.items():
    md = data.get("metadata", {})
    records.append({
        "url": url,
        "title": md.get("title", ""),
        "description": md.get("description", ""),
        "main_heading": md.get("main_heading", ""),
        "scraped_at": md.get("scraped_at", None),
        "status": data.get("status", ""),
        "content_md": data.get("content", ""),
    })

df = pd.DataFrame.from_records(records)
output_path = site_data_dir / "mobibikes_ca_content.parquet"
save_to_parquet(df, output_path)

## Load and Explore

In [0]:
# Load
trips = pd.read_parquet(trip_data_dir / "mobi_trips.parquet")
stations = pd.read_parquet(trip_data_dir / "mobi_stations.parquet")
mobi_site = pd.read_parquet(site_data_dir / "mobibikes_ca_content.parquet")

print(f"Trips: {len(trips):,}")
print(f"Stations: {len(stations)}")
print(f"Date range: {trips['departure_time'].min()} to {trips['departure_time'].max()}")
print(f"Site pages scraped: {len(mobi_site)}")

## Write out bronze tables

In [0]:
catalog = dbutils.widgets.get('catalog')
schema = dbutils.widgets.get('schema')

import pyspark.sql.functions as F

# Use /Volumes paths directly for Spark IO
trips_pq = str(trip_data_dir / 'mobi_trips.parquet')
stations_pq = str(trip_data_dir / 'mobi_stations.parquet')
site_pq = str(site_data_dir / 'mobibikes_ca_content.parquet')

# Trips: partition by year/month for better performance
trips_sdf = (
    spark.read.parquet(trips_pq)
    .withColumn('year', F.year('departure_time'))
    .withColumn('month', F.month('departure_time'))
    .repartition('year', 'month')
)

(
  trips_sdf.write.format('delta').mode('overwrite')
  .option('overwriteSchema', True)
  .partitionBy('year', 'month')
  .saveAsTable(f'`{catalog}`.`{schema}`.`bronze_trips`')
)

(
  spark.read.parquet(stations_pq)
  .coalesce(1)
  .write.format('delta')
  .mode('overwrite')
  .option('overwriteSchema', True)
  .saveAsTable(f'`{catalog}`.`{schema}`.`bronze_stations`')
)

(
  spark.read.parquet(site_pq)
  .coalesce(1)
  .write.format('delta')
  .mode('overwrite')
  .option('overwriteSchema', True)
  .saveAsTable(f'`{catalog}`.`{schema}`.`bronze_site`')
)

In [0]:
## Create Silver Trips


In [0]:
# Silver: match trips by either station name OR station_id + ' ' + name (fast, broadcast, escaped)
catalog = dbutils.widgets.get('catalog')
schema = dbutils.widgets.get('schema')

stations_sql = f"""
CREATE OR REPLACE TEMP VIEW stations_keys AS
SELECT
  CAST(station_id AS STRING) AS station_id,
  lower(regexp_replace(trim(name), '\\s+', ' ')) AS name_norm,
  lower(regexp_replace(trim(concat(CAST(station_id AS STRING), ' ', name)), '\\s+', ' ')) AS code_plus_name_norm
FROM `{catalog}`.`{schema}`.`bronze_stations`;
"""

silver_sql = f"""
CREATE OR REPLACE TABLE `{catalog}`.`{schema}`.`silver_trips`
USING delta AS
SELECT /*+ BROADCAST(sd_name), BROADCAST(sd_code), BROADCAST(sr_name), BROADCAST(sr_code) */
  b.*,
  coalesce(sd_name.station_id, sd_code.station_id) AS departure_station_id,
  coalesce(sr_name.station_id, sr_code.station_id) AS return_station_id
FROM `{catalog}`.`{schema}`.`bronze_trips` b
LEFT JOIN stations_keys sd_name
  ON lower(regexp_replace(trim(b.departure_station_name), '\\s+', ' ')) = sd_name.name_norm
LEFT JOIN stations_keys sd_code
  ON lower(regexp_replace(trim(b.departure_station_name), '\\s+', ' ')) = sd_code.code_plus_name_norm
LEFT JOIN stations_keys sr_name
  ON lower(regexp_replace(trim(b.return_station_name), '\\s+', ' ')) = sr_name.name_norm
LEFT JOIN stations_keys sr_code
  ON lower(regexp_replace(trim(b.return_station_name), '\\s+', ' ')) = sr_code.code_plus_name_norm;
"""

spark.sql(stations_sql)
spark.sql(silver_sql)

print("✓ silver_trips created")


In [0]:
# Quick QA: unmatched stations after join
catalog = dbutils.widgets.get('catalog')
schema = dbutils.widgets.get('schema')

display(spark.sql(f"""
SELECT
  SUM(CASE WHEN departure_station_id IS NULL THEN 1 ELSE 0 END) AS dep_unmatched,
  SUM(CASE WHEN return_station_id    IS NULL THEN 1 ELSE 0 END) AS ret_unmatched
FROM `{catalog}`.`{schema}`.`silver_trips`
"""))
